<a href="https://colab.research.google.com/github/ian-mcnair/ForageSnap/blob/master/MNV2_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Statements

Installs all needed packages.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

     |████████████████████████████████| 378.9MB 44kB/s 
     |████████████████████████████████| 61kB 22.0MB/s 
     |████████████████████████████████| 450kB 41.3MB/s 
     |████████████████████████████████| 4.3MB 31.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Creating Image Generators

Base directory is where all the files are located in our shared Google Drive

In [0]:
base_dir = '/content/drive/My Drive/Project FORSCAN/Datasets/PoC Images'
base_dir

'/content/drive/My Drive/Project FORSCAN/Datasets/PoC Images'

In [0]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split = 0.2,
    rotation_range = 45,
    horizontal_flip = True,
    vertical_flip = True,
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training',
    shuffle = True,
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation',
    shuffle = True,
)

Found 7112 images belonging to 14 classes.
Found 1769 images belonging to 14 classes.


In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 14))

# Making the Labels.txt File for Later

In [0]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Edible: Alligator Juniper': 0, 'Edible: Asparagus': 1, 'Edible: Bamboo': 2, 'Edible: Berries': 3, 'Edible: Dandelion': 4, 'Edible: Pecans': 5, 'Edible: Rosemary': 6, 'Edible: Southern Magnolia': 7, 'Harmful: Laurel Family': 8, 'Harmful: Nightshade': 9, 'Harmful: Poison Ivy': 10, 'Harmful: Poison Oak': 11, 'Harmful: Poison Sumac': 12, 'Harmful: Yew': 13}


In [0]:
!cat labels.txt

Edible: Alligator Juniper
Edible: Asparagus
Edible: Bamboo
Edible: Berries
Edible: Dandelion
Edible: Pecans
Edible: Rosemary
Edible: Southern Magnolia
Harmful: Laurel Family
Harmful: Nightshade
Harmful: Poison Ivy
Harmful: Poison Oak
Harmful: Poison Sumac
Harmful: Yew

# Creating the Pretrained Model

In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                          include_top = False,
                                          weights = 'imagenet')


9412608/9406464 [==============================] - 0s 0us/step


You can make the model trainable of freeze it. You can also specifiy at which layer you want to enable training

In [0]:
model.trainable = True

# Recreating the Output Layer
num_classes should match number of classes

In [0]:
num_classes = 14

model = tf.keras.Sequential([
  model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(num_classes, activation='softmax') # The layer amount needs to match the classes
])

# Compiling The Model


In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-4, decay = 1e-5 ), # 1e-4
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
# model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 160


# Setting Up Checkpoints to save the model as it trains

In [0]:
# This currently causes a known error when trying to retrain
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="/content/drive/My Drive/Project FORSCAN/Models/Checkpoints/MBNV2CheckpointsIan1e-5Decay-7.30-10am9pm-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')
callbacks_list = [checkpoint]

# Loading a Model instead of creating a new one

In [0]:
# Use only when needed
# Doesn't work. Raises errors about needing a placeholder
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/Project FORSCAN/Models/Checkpoints/MBNV2CheckpointsIan1e-5Decay-7.30-10am9pm-09-0.75.hdf5')

W0731 13:41:52.055122 140562483439488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1423: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0731 13:42:18.652628 140562483439488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


In [0]:
# model.weights

# Training

In [0]:
epochs = 15

history = model.fit_generator(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator,
                    callbacks = callbacks_list,
                    shuffle = True
)

Epoch 1/15
111/112 [============================>.] - ETA: 22s - loss: 0.2190 - accuracy: 0.9268
Epoch 00001: val_accuracy improved from -inf to 0.70831, saving model to /content/drive/My Drive/Project FORSCAN/Models/Checkpoints/MBNV2CheckpointsIan1e-5Decay-7.30-10am9pm-01-0.71.hdf5
112/112 [==============================] - 3133s 28s/step - loss: 0.2211 - accuracy: 0.9258 - val_loss: 1.4008 - val_accuracy: 0.7083
Epoch 2/15
111/112 [============================>.] - ETA: 20s - loss: 0.1910 - accuracy: 0.9364
Epoch 00002: val_accuracy did not improve from 0.70831
112/112 [==============================] - 2440s 22s/step - loss: 0.1909 - accuracy: 0.9366 - val_loss: 1.6468 - val_accuracy: 0.6750
Epoch 3/15
111/112 [============================>.] - ETA: 20s - loss: 0.1905 - accuracy: 0.9407
Epoch 00003: val_accuracy improved from 0.70831 to 0.71792, saving model to /content/drive/My Drive/Project FORSCAN/Models/Checkpoints/MBNV2CheckpointsIan1e-5Decay-7.30-10am9pm-03-0.72.hdf5
112/112 [

# Accuracy Histories

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
# plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
# Saving, Exporting, and Downloading Files needed for Android

In [0]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')